In [55]:
import pandas as pd

In [56]:
data=pd.read_csv("/content/drive/My Drive/mobile.csv")

In [57]:
data.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,6,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,6,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,9,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,14,1208,1212,1411,8,2,15,1,1,0,1


In [58]:
data["price_range"].value_counts()

3    500
2    500
1    500
0    500
Name: price_range, dtype: int64

In [59]:
X=data.drop("price_range",axis=1)
Y=data["price_range"]

In [ ]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [ ]:
X_train.shape

(1600, 20)

In [60]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [77]:
## Perform Hyperparameter Optimization
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [63]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization, Dropout
from keras.activations import relu, sigmoid

In [64]:
def create_model(layers, activation):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i==0: # for first layer  
            model.add(Dense(nodes,input_dim=X_train.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
        else: # for other layer 
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
     # output layer        
    model.add(Dense(units = 4, kernel_initializer= 'glorot_uniform', activation = 'sigmoid')) 
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    return model


In [67]:
model = KerasClassifier(build_fn=create_model, verbose=0)

In [79]:
layers = [(20), (40, 20), (45, 30, 15)] # number of layers = 3 & number of nodes inside [40,20]= 2 layers with 40 neron for first and 3  for second 
activations = ['sigmoid', 'relu']

In [84]:
param_grid = dict(layers=layers, activation=activations, batch_size = [100,256], epochs=[30])
grid = GridSearchCV(estimator=model, param_grid=param_grid,cv=5)

In [85]:
grid_result = grid.fit(X_train, y_train)


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
TypeError: 'int' object is not iterable

  FitFailedWarning)


In [82]:
[grid_result.best_score_,grid_result.best_params_]


[0.7112499952316285,
 {'activation': 'sigmoid',
  'batch_size': 128,
  'epochs': 30,
  'layers': (40, 20)}]